# TFM Suministro Global - Materiales - Neo4j - py2neo 

## Partimos de la existencia, en paralelo a esta ejecución, de una instancia levantada y accesible de Neo4j  
### La primera vez que se quiera usar _py2neo_ erá  necesario asegurar su instalación:  _!pip install py2neo_

In [1]:
##!pip install py2neo 

In [2]:
#importar la librería para poder hacer uso de sus funcionalidades y objetos
from py2neo import *

In [3]:
#acceder a la instancia levantada de Neo4j - en nuestro caso instancia Docker - asignamos credenciales de autentificación -
graph_test01 = Graph(auth=('neo4j', 's3cr3t'))

In [4]:
#que es el objeto - graph_test01
graph_test01

Graph('bolt://localhost:7687')

In [5]:
#que es el objeto - tx
tx = graph_test01.begin()

In [6]:
tx

In [7]:
#query de conexión de prueba sin errores
queryPRB= "MATCH (n) RETURN n LIMIT 2"
graph_test01.run(queryPRB)

n
"(_0:MaterialB {Mat_Abrev: 'GBrt', Mat_Familia: 'General', Mat_Subfamilia: 'Industrial & constr. minerals', name: 'Baryte'})"
"(_1:MaterialB {Mat_Abrev: 'Bx', Mat_Familia: 'General', Mat_Subfamilia: 'Other non-ferrous metals', name: 'Bauxite'})"


## CARGAR DATOS

In [8]:
import numpy as np    #datos y operativa álgegra y matrices de n dimensiones (arrays)
import pandas as pd   #carga dataset y manipulación de datos  y dataframes

In [9]:
#DATOS: Fichero completo de MATERIAS PRIMAS CRÍTICAS en el suministro GLOBAL - Atendiendo al alcance del estudio - EU CRM 2020
#FICHERO: GLMateriales_cargar.csv
#CONTENIDO: Materiales 
#USO: Crear los NODOS de tipo "MaterialB" - Material bruto y "MaterialREF" - Material Refinado) 
## -- propiedades de cada material: nombre, abreviatura, familia y subfamilia 
## -- los datos de suministro -posteriores- fijarán las relaciones entre materiales, países proveedores y usos del material 
rutaFichero =  "https://raw.githubusercontent.com/JuanKFD/UNIR_TFM/d10cd756788ce5e4b93f6b5c7ca4d40cb4aed297/GLMateriales_cargar.csv" 
dfMateriales = pd.read_csv(rutaFichero, sep=';')


In [10]:
dfMateriales.head()

,Material,Mat_Abrev,Mat_Familia,Mat_Subfamilia
0,Aluminium,Al,General,Other non-ferrous metals
1,Antimony,Sb,General,Other non-ferrous metals
2,Arsenic,As,General,Other non-ferrous metals
3,Baryte,GBrt,General,Industrial & constr. minerals
4,Bauxite,Bx,General,Other non-ferrous metals


In [11]:
dfMateriales.tail()

,Material,Mat_Abrev,Mat_Familia,Mat_Subfamilia
73,Tungsten,W,General,Iron and ferro-alloy metals
74,Vanadium,V,General,Iron and ferro-alloy metals
75,Yttrium,Y,Rare earths,HREEs-Heavy
76,Zinc,Zn,General,Other non-ferrous metals
77,Zirconium,Zr,General,Other non-ferrous metals


In [12]:
dfMateriales.shape

(78, 4)

In [13]:
dfMateriales.shape[0]

78

In [14]:
dfMateriales.columns

Index(['Material', 'Mat_Abrev', 'Mat_Familia', 'Mat_Subfamilia'], dtype='object')

In [15]:
nFilas = dfMateriales.shape[0]

In [16]:
nFilas

78

# PREVISUALIZACION DE DATOS - MATERIALES
a = Node("Material", name=dfMateriales.Material[i])

In [17]:
for i in range(0, nFilas):
    print(dfMateriales.Material[i])
    print(dfMateriales.Mat_Abrev[i])
    print(dfMateriales.Mat_Familia[i])
    print(dfMateriales.Mat_Subfamilia[i])

Aluminium
Al
General
Other non-ferrous metals
Antimony
Sb
General
Other non-ferrous metals
Arsenic
As
General
Other non-ferrous metals
Baryte
GBrt
General
Industrial & constr. minerals
Bauxite
Bx
General
Other non-ferrous metals
Bentonite
GBn
General
Industrial & constr. minerals
Beryllium
Be
General
Other non-ferrous metals
Bismuth
Bi
General
Other non-ferrous metals
Borate
Gbo
General
Industrial & constr. minerals
Cadmium
Cd
General
Other non-ferrous metals
Cerium
Ce
Rare earths
LREEs-Light
Chromium
Cr
General
Iron and ferro-alloy metals
Cobalt
Co
General
Iron and ferro-alloy metals
Coking coal
GCc
General
Bio and other materials
Copper
Cu
General
Other non-ferrous metals
Diatomite
Di
General
Industrial & constr. minerals
Dysprosium
Dy
Rare earths
HREEs-Heavy
Erbium
Er
Rare earths
HREEs-Heavy
Europium
Eu
Rare earths
HREEs-Heavy
Feldspar
Fsp
General
Industrial & constr. minerals
Fluorspar
Fl
General
Industrial & constr. minerals
Gadolinium
Gd
Rare earths
HREEs-Heavy
Gallium
Ga
General

In [18]:
for colName in dfMateriales.columns:
    print(colName)

Material
Mat_Abrev
Mat_Familia
Mat_Subfamilia


In [19]:
dfMateriales.columns[0]

'Material'

In [23]:
#  Crear NODO DE MATERIAL por cada fila junto con sus PROPIEDADES
#  Nodos: MATERIALES -- "MaterialB" bruto ; "MRefinado" refinado
## PREVISAULIZAR QUERYs antes de la ejecución
TipoNodo = " :MaterialREF " 
for i in range(0, nFilas):
    strQry = "CREATE (m " + TipoNodo + "{"                                                             \
        + "name: '" + dfMateriales.Material[i]       + "_REF', "                                       \
        + dfMateriales.columns[1] + ": " + "'" + dfMateriales.Mat_Abrev[i]  + "', "                    \
        + dfMateriales.columns[2] + ": " + "'" + dfMateriales.Mat_Familia[i]    + "', "                \
        + dfMateriales.columns[3] + ": " + "'" + dfMateriales.Mat_Subfamilia[i] + "' })"               \
        + " RETURN m" 
    print("QRY [" + str(i) + "]> "+ strQry)
# querys  visualizadas

QRY [0]> CREATE (m  :MaterialREF {name: 'Aluminium_REF', Mat_Abrev: 'Al', Mat_Familia: 'General', Mat_Subfamilia: 'Other non-ferrous metals' }) RETURN m
QRY [1]> CREATE (m  :MaterialREF {name: 'Antimony_REF', Mat_Abrev: 'Sb', Mat_Familia: 'General', Mat_Subfamilia: 'Other non-ferrous metals' }) RETURN m
QRY [2]> CREATE (m  :MaterialREF {name: 'Arsenic_REF', Mat_Abrev: 'As', Mat_Familia: 'General', Mat_Subfamilia: 'Other non-ferrous metals' }) RETURN m
QRY [3]> CREATE (m  :MaterialREF {name: 'Baryte_REF', Mat_Abrev: 'GBrt', Mat_Familia: 'General', Mat_Subfamilia: 'Industrial & constr. minerals' }) RETURN m
QRY [4]> CREATE (m  :MaterialREF {name: 'Bauxite_REF', Mat_Abrev: 'Bx', Mat_Familia: 'General', Mat_Subfamilia: 'Other non-ferrous metals' }) RETURN m
QRY [5]> CREATE (m  :MaterialREF {name: 'Bentonite_REF', Mat_Abrev: 'GBn', Mat_Familia: 'General', Mat_Subfamilia: 'Industrial & constr. minerals' }) RETURN m
QRY [6]> CREATE (m  :MaterialREF {name: 'Beryllium_REF', Mat_Abrev: 'Be', Mat

In [24]:
graph_test01

Graph('bolt://localhost:7687')

In [25]:
# Crear un nodo por cada fila con sus propiedades
#nodos de MATERIALES,
# CREAR NODOS: "MaterialB" 
## query para crea nodo con sus  propiedades
TipoNodo = " :MaterialREF " 
for i in range(0, nFilas):
    strQry = "CREATE (m " + TipoNodo + "{"                                                             \
        + "name: '" + dfMateriales.Material[i]       + "_REF', "                                       \
        + dfMateriales.columns[1] + ": " + "'" + dfMateriales.Mat_Abrev[i]  + "', "                    \
        + dfMateriales.columns[2] + ": " + "'" + dfMateriales.Mat_Familia[i]    + "', "                \
        + dfMateriales.columns[3] + ": " + "'" + dfMateriales.Mat_Subfamilia[i] + "' })"               \
        + " RETURN m" 
    graph_test01.run(strQry)
    print("EJECUTADA QRY [" + str(i) + "]> "+ strQry)
# nodos creados

EJECUTADA QRY [0]> CREATE (m  :MaterialREF {name: 'Aluminium_REF', Mat_Abrev: 'Al', Mat_Familia: 'General', Mat_Subfamilia: 'Other non-ferrous metals' }) RETURN m
EJECUTADA QRY [1]> CREATE (m  :MaterialREF {name: 'Antimony_REF', Mat_Abrev: 'Sb', Mat_Familia: 'General', Mat_Subfamilia: 'Other non-ferrous metals' }) RETURN m
EJECUTADA QRY [2]> CREATE (m  :MaterialREF {name: 'Arsenic_REF', Mat_Abrev: 'As', Mat_Familia: 'General', Mat_Subfamilia: 'Other non-ferrous metals' }) RETURN m
EJECUTADA QRY [3]> CREATE (m  :MaterialREF {name: 'Baryte_REF', Mat_Abrev: 'GBrt', Mat_Familia: 'General', Mat_Subfamilia: 'Industrial & constr. minerals' }) RETURN m
EJECUTADA QRY [4]> CREATE (m  :MaterialREF {name: 'Bauxite_REF', Mat_Abrev: 'Bx', Mat_Familia: 'General', Mat_Subfamilia: 'Other non-ferrous metals' }) RETURN m
EJECUTADA QRY [5]> CREATE (m  :MaterialREF {name: 'Bentonite_REF', Mat_Abrev: 'GBn', Mat_Familia: 'General', Mat_Subfamilia: 'Industrial & constr. minerals' }) RETURN m
EJECUTADA QRY [6]

EJECUTADA QRY [56]> CREATE (m  :MaterialREF {name: 'Rhodium_REF', Mat_Abrev: 'Rh', Mat_Familia: 'Precious metals', Mat_Subfamilia: 'PGM-Platinum Group Metals' }) RETURN m
EJECUTADA QRY [57]> CREATE (m  :MaterialREF {name: 'Ruthenium_REF', Mat_Abrev: 'Ru', Mat_Familia: 'Precious metals', Mat_Subfamilia: 'PGM-Platinum Group Metals' }) RETURN m
EJECUTADA QRY [58]> CREATE (m  :MaterialREF {name: 'Samarium_REF', Mat_Abrev: 'Sm', Mat_Familia: 'Rare earths', Mat_Subfamilia: 'LREEs-Light' }) RETURN m
EJECUTADA QRY [59]> CREATE (m  :MaterialREF {name: 'Sapele wood_REF', Mat_Abrev: 'GSw', Mat_Familia: 'General', Mat_Subfamilia: 'Bio and other materials' }) RETURN m
EJECUTADA QRY [60]> CREATE (m  :MaterialREF {name: 'Scandium_REF', Mat_Abrev: 'Sc', Mat_Familia: 'Rare earths', Mat_Subfamilia: 'HREEs-Heavy' }) RETURN m
EJECUTADA QRY [61]> CREATE (m  :MaterialREF {name: 'Selenium_REF', Mat_Abrev: 'Se', Mat_Familia: 'General', Mat_Subfamilia: 'Other non-ferrous metals' }) RETURN m
EJECUTADA QRY [62]>

In [26]:
dfGraphMat= graph_test01.run("MATCH (n :MaterialREF ) RETURN n").data()

In [27]:
dfGraphMat[0]


{'n': Node('MaterialREF', Mat_Abrev='Al', Mat_Familia='General', Mat_Subfamilia='Other non-ferrous metals', name='Aluminium_REF')}

In [28]:
graph_test01.run("MATCH (n :MaterialREF) RETURN DISTINCT LABELS(n), COUNT(n)")

LABELS(n),COUNT(n)
['MaterialREF'],78


In [29]:
graph_test01.run("MATCH (n :MaterialB) RETURN DISTINCT LABELS(n), COUNT(n)")

LABELS(n),COUNT(n)
['MaterialB'],78


# fin proceso - MATERIALES REFINADOS -y brutos- CREADOS

In [8]:
# fin archivo